# PARTIE 3 : Données sur les hôtels

## Import librairies

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import json

## Lecture du fichier json

In [3]:
file = open('./src/booking.json')
results = json.load(file)
print(type(results))
print(type(results[0]))

<class 'list'>
<class 'dict'>


In [71]:
data = pd.DataFrame(results)
print(data.shape)
data.head()

(698, 8)


,city_id,hotel_name,description,score,price,url,lat_lon
0,6,Hotel des Vosges,L'Hotel des Vosges vous accueille dans le XXe ...,"8,0",\n€ 102\n,https://www.booking.com/hotel/fr/des-vosges-pa...,"48.86777797,2.38290846"
1,1,Le Héron,"Offrant une vue sur la rue calme, Le Héron est...","8,5",\n€ 137\n,https://www.booking.com/hotel/fr/le-heron-huis...,"48.61008410,-1.45673760"
2,1,Les Terrasses Poulard,Occupant 2 bâtiments différents au cœur du Mon...,"7,4",\n€ 242\n,https://www.booking.com/hotel/fr/les-terrasses...,"48.63534943,-1.51037872"
3,5,Les Initiés,Installé au cœur du centre historique de Rouen...,"7,9",\n€ 81\n,https://www.booking.com/hotel/fr/les-inities-r...,"49.44197641,1.09243892"
4,6,Ideal Hotel,"Situé à Paris, l'Ideal Hotel propose des héber...","5,3",\n€ 75\n,https://www.booking.com/hotel/fr/ideal-paris.f...,"48.86761040,2.37546130"


## Nettoyage des données

#### Séparer latitude et longitude en 2 colonnes distinctes

In [72]:
data['latlon'] = data['lat_lon'].str.split(',')
data2 = pd.DataFrame(data['latlon'].tolist(), columns=['lat', 'lon'], index=data.index)

hotels_df = pd.concat([data, data2], axis=1)
hotels_df.drop(['lat_lon', 'latlon'], axis=1, inplace=True)
hotels_df.head()

,city_id,hotel_name,description,score,price,url,lat,lon
0,6,Hotel des Vosges,L'Hotel des Vosges vous accueille dans le XXe ...,"8,0",\n€ 102\n,https://www.booking.com/hotel/fr/des-vosges-pa...,48.86777797,2.38290846
1,1,Le Héron,"Offrant une vue sur la rue calme, Le Héron est...","8,5",\n€ 137\n,https://www.booking.com/hotel/fr/le-heron-huis...,48.61008410,-1.45673760
2,1,Les Terrasses Poulard,Occupant 2 bâtiments différents au cœur du Mon...,"7,4",\n€ 242\n,https://www.booking.com/hotel/fr/les-terrasses...,48.63534943,-1.51037872
3,5,Les Initiés,Installé au cœur du centre historique de Rouen...,"7,9",\n€ 81\n,https://www.booking.com/hotel/fr/les-inities-r...,49.44197641,1.09243892
4,6,Ideal Hotel,"Situé à Paris, l'Ideal Hotel propose des héber...","5,3",\n€ 75\n,https://www.booking.com/hotel/fr/ideal-paris.f...,48.86761040,2.37546130


#### Corriger le champ prix

In [73]:
for index, value in enumerate(hotels_df['price']):
    hotels_df.iloc[index,4] = value.split()[1]

#### Mettre toutes les données numériques au bon format

In [74]:
hotels_df['city_id'] = hotels_df['city_id'].astype('int')
hotels_df['score'] = hotels_df['score'].str.replace(',','.').astype('float')
hotels_df['price'] = hotels_df['price'].astype('int')
hotels_df['lat'] = hotels_df['lat'].astype('float')
hotels_df['lon'] = hotels_df['lon'].astype('float')

In [75]:
hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city_id      698 non-null    int64  
 1   hotel_name   698 non-null    object 
 2   description  698 non-null    object 
 3   score        689 non-null    float64
 4   price        698 non-null    int64  
 5   url          698 non-null    object 
 6   lat          698 non-null    float64
 7   lon          698 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 43.8+ KB


#### Enregistrement fichier csv

In [93]:
hotels_df.to_csv('./src/hotels.csv', index=False)

In [ ]:
hotels_df = pd.read_csv('./src/hotels.csv')

## Ajouter les hôtels dans le dataframe villes

In [77]:
cities_df = pd.read_csv('./src/top_35_cities_lite.csv')

In [79]:
cities_df['hotels'] = None
for index, row in cities_df.iterrows():
    hotels_list = hotels_df[hotels_df['city_id'] == row['city_id']].to_dict(orient='records')
    cities_df.at[index, 'hotels'] = hotels_list
cities_df

,city_id,city,lat_city,lon_city,avg_weather,avg_temp,avg_clouds,avg_wind,avg_humidity,avg_rain,avg_pop,hotels
0,1,Mont Saint Michel,48.635954,-1.511460,Rain,5.668,82.6,7.708,87.0,0.446,0.528,"[{'city_id': 1, 'hotel_name': 'Le Héron', 'des..."
1,2,St Malo,48.649518,-2.026041,Rain,6.324,84.0,8.852,84.0,0.524,0.546,"[{'city_id': 2, 'hotel_name': 'Brit Hotel Le S..."
2,3,Bayeux,49.276462,-0.702474,Rain,5.518,71.2,6.992,88.8,0.832,0.616,"[{'city_id': 3, 'hotel_name': 'Mon Saint-Malo ..."
3,4,Le Havre,49.493898,0.107973,Rain,7.252,74.2,9.022,82.0,0.806,0.646,"[{'city_id': 4, 'hotel_name': 'Eklo Hotels Le ..."
4,5,Rouen,49.440459,1.093966,Rain,5.518,80.2,6.328,87.6,0.590,0.528,"[{'city_id': 5, 'hotel_name': 'Les Initiés', '..."
5,6,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664,"[{'city_id': 6, 'hotel_name': 'Hotel des Vosge..."
6,7,Amiens,49.894171,2.295695,Rain,5.940,87.8,6.220,86.8,0.752,0.622,"[{'city_id': 7, 'hotel_name': 'La Parenthèse E..."
7,8,Lille,50.636565,3.063528,Rain,5.448,93.8,6.932,83.0,1.536,0.602,"[{'city_id': 8, 'hotel_name': 'Le Chat Qui Dor..."
8,9,Strasbourg,48.584614,7.750713,Clouds,5.882,77.4,4.144,80.8,0.478,0.410,"[{'city_id': 9, 'hotel_name': 'Hotel Victoria'..."
9,10,Chateau du Haut Koenigsbourg,48.249523,7.345492,Clouds,2.034,79.0,3.320,86.4,0.860,0.508,"[{'city_id': 10, 'hotel_name': 'Relais et Chât..."


In [92]:
# Afficher le premier hôtel pour la première ville
cities_df.iloc[0,-1][0]

{'city_id': 1,
 'hotel_name': 'Le Héron',
 'description': "Offrant une vue sur la rue calme, Le Héron est un hébergement situé à Huisnes-sur-Mer, à 12 km du Mont-Saint-Michel et à 19 km du Scriptorial d'Avranches, musee des manuscrits. du Mont Saint-Michel. Vous bénéficierez gratuitement d'un parking privé et d'une connexion Wi-Fi. Cet établissement non-fumeurs se trouve à 12 km de l'abbaye du Mont-Saint-Michel.\n\nCette maison de vacances comprend 3 chambres ainsi qu'une cuisine équipée d'un lave-vaisselle et d'un four. Il comprend une télévision à écran plat, un coin salon et 2 salles de bains pourvues d'une douche à l'italienne. Les serviettes et le linge de lit sont fournis. L'établissement possède un coin repas extérieur.\n\nVous séjournerez à 44 km de la gare de Granville et à 45 km du zoo de Champrepus. L'aéroport de Rennes-Saint-Jacques, le plus proche, est à 92 km.",
 'score': 8.5,
 'price': 137,
 'url': 'https://www.booking.com/hotel/fr/le-heron-huisnes-sur-mer.fr.html',
 'la

## Enregistrement dataframe final

In [94]:
cities_df.to_csv('./src/top_35_cities_with_hotels.csv', index=False)

## Top 20 hôtels des 5 meilleures destinations

In [124]:
best_destinations_df = pd.read_csv('./src/top_5_destinations.csv')

In [119]:
best_destinations_id = best_destinations['city_id'].values.tolist()
best_destinations_id

[6, 10, 16, 18, 30]

In [147]:
best_hotels_df = hotels_df[hotels_df['city_id'].isin(best_destinations_id)]
best_hotels_df.reset_index(drop = True, inplace = True)
best_hotels_df.shape

(100, 8)

In [149]:
best_hotels_df.isna().sum()

city_id        0
hotel_name     0
description    0
score          1
price          0
url            0
lat            0
lon            0
dtype: int64

In [150]:
best_hotels_df[best_hotels_df['score'].isna() == True]

,city_id,hotel_name,description,score,price,url,lat,lon
86,30,Maison 5 chambres,"La Maison 5 chambres est située à Bélesta, à 1...",NaN,178,https://www.booking.com/hotel/fr/maison-5-cham...,42.898085,1.930289


In [153]:
best_hotels_df.iloc[86,3] = 0

In [154]:
top_hotels_df = pd.merge(best_hotels_df, best_destinations_df, left_on='city_id', right_on='city_id')
top_hotels_df

,city_id,hotel_name,description,score,price,url,lat,lon,city,lat_city,lon_city,avg_weather,avg_temp,avg_clouds,avg_wind,avg_humidity,avg_rain,avg_pop
0,6,Hotel des Vosges,L'Hotel des Vosges vous accueille dans le XXe ...,8.0,102,https://www.booking.com/hotel/fr/des-vosges-pa...,48.867778,2.382908,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664
1,6,Ideal Hotel,"Situé à Paris, l'Ideal Hotel propose des héber...",5.3,75,https://www.booking.com/hotel/fr/ideal-paris.f...,48.867610,2.375461,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664
2,6,Printania Porte de Versailles,Situé à 900 mètres du parc des expositions de ...,5.7,87,https://www.booking.com/hotel/fr/printaniasaph...,48.834068,2.295335,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664
3,6,Résidence de Bourgogne,Dotée d'un bar et offrant une vue sur la ville...,6.7,103,https://www.booking.com/hotel/fr/residence-de-...,48.869827,2.355760,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664
4,6,Hotel du Pré,"Le charmant 3 étoiles Hôtel du Pré, où le géra...",8.3,159,https://www.booking.com/hotel/fr/hoteldupre.fr...,48.877522,2.347065,Paris,48.858890,2.320041,Rain,6.874,93.8,5.798,77.6,1.268,0.664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,30,Hôtel Terranostra,"Doté d'un restaurant et d'un bar, l'Hotel Terr...",8.5,91,https://www.booking.com/hotel/fr/terranostra.f...,42.847186,1.605603,Ariege,42.992743,1.612497,Rain,5.834,96.6,2.742,86.0,1.088,0.630
96,30,"The Originals Access, Hôtel Foix (P'tit Dej-Ho...",Situé au sein du parc naturel régional des Pyr...,8.0,64,https://www.booking.com/hotel/fr/campanilefoix...,42.992112,1.615832,Ariege,42.992743,1.612497,Rain,5.834,96.6,2.742,86.0,1.088,0.630
97,30,La Grange - 10 couchages,"Situé à Carla-de-Roquefort, à 20 km du château...",9.3,215,https://www.booking.com/hotel/fr/la-grange-car...,42.975199,1.765422,Ariege,42.992743,1.612497,Rain,5.834,96.6,2.742,86.0,1.088,0.630
98,30,Chalet bois au milieu des Pyrénées,Doté d'une terrasse et d'une réception ouverte...,9.0,130,https://www.booking.com/hotel/fr/chalet-bois-a...,42.589904,1.798368,Ariege,42.992743,1.612497,Rain,5.834,96.6,2.742,86.0,1.088,0.630


## Visualisation sur carte

In [168]:
import plotly.express as px

fig = px.scatter_mapbox(top_hotels_df, 
                        lat = "lat", lon = "lon", 
                        color = "avg_temp",
                        size = "score",
                        hover_name = "hotel_name",
                        hover_data = {"lat": False, "lon": False, "score": ":.1f", "price": True, "city": True, "avg_temp": ":.0f"},
                        mapbox_style ="carto-positron",
                        zoom = 4,
                        center = {'lat':46.232192999999995,'lon':2.209666999999996},
                        color_continuous_scale = px.colors.sequential.Bluered
                        )
fig.update_layout(title_text="Top 20 hotels for the 5 best French destinations", title_x=0.5)
fig.show()